# 杭州萧山区项目
## 本地排放清单预处理`Local Emission Inventory`

---
*@author: Evan*\
*@date: 2023-07-15*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../../src/')
from namelist import *
import findpoint as fp

In [2]:
grid = xr.open_dataset(datadir+'GRIDCRO2D_2022234.nc')
lat = grid.LAT[0,0,:,:]
lon = grid.LON[0,0,:,:]

gridfile = xr.Dataset(
    data_vars = dict(
        ShapeVar = (['y','x'],np.zeros_like(lat),{'long name':'not-used variable'})
    ),
    coords=dict(
        latitude = (['y','x'],lat.data),
        longitude = (['y','x'],lon.data)
    )
)
gridfile

<xarray.Dataset>
Dimensions:    (y: 138, x: 135)
Coordinates:
    latitude   (y, x) float32 28.29 28.29 28.29 28.29 ... 31.89 31.89 31.89
    longitude  (y, x) float32 117.9 117.9 117.9 118.0 ... 122.3 122.3 122.4
Dimensions without coordinates: y, x
Data variables:
    ShapeVar   (y, x) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [25]:
ef = pd.ExcelFile(emisarea)
ef.sheet_names

['机动车',
 '非道路-船舶和港口机械',
 '非道路-工程机械',
 '非道路-农业机械',
 '扬尘源-道路扬尘',
 '扬尘源-施工扬尘',
 '扬尘源-土壤扬尘',
 '非工业溶剂使用-农药使用',
 '非工业溶剂使用-建筑涂料使用',
 '非工业溶剂使用-其他',
 '生物质-露天焚烧',
 '生物质-家用秸秆薪柴',
 '农业源-畜禽养殖',
 '农业源-氮肥施用',
 '农业源-土壤本底',
 '农业源-固氮植物',
 '农业源-秸秆堆肥',
 '农业源-人体粪便',
 '其他排放源-餐饮',
 '其他排放源-民用燃烧',
 '面源合计']

In [31]:
species = ['SO2','NOx','CO','PM10','PM2.5','VOCs','NH3','BC','OC']
dfs={}

for sheet_name in ef.sheet_names:
    parts = sheet_name.split('-')
    shared_part = parts[0]
    unique_part = parts[1] if len(parts) > 1 else None
    
    current_df = ef.parse(sheet_name).drop(['FID_1'],axis=1)
    coord_cols = ['CenLon', 'CenLat']

    if shared_part not in dfs:
        dfs[shared_part] = current_df
    else:
        merged_df = pd.merge(dfs[shared_part], current_df, on=coord_cols,
                             how='outer', suffixes=('_x', '_y'))

        for specie in species:
            merged_df[specie] = merged_df[f'{specie}_x'] + merged_df[f'{specie}_y']
            merged_df = merged_df.drop(columns=[f'{specie}_x',f'{specie}_y'],axis=1)
        
        dfs[shared_part] = merged_df

In [34]:
dfs.keys()

dict_keys(['机动车', '非道路', '扬尘源', '非工业溶剂使用', '生物质', '农业源', '其他排放源', '面源合计'])

In [35]:
sections = ['机动车', '非道路', '扬尘源', '非工业溶剂使用',
            '生物质', '农业源', '其他排放源', '面源合计']
secnames = ['motor','non-road','dust','non-ind_solvent',
            'biomass','agriculture','residential','total']

for sec, secname in zip(sections,secnames):
    temp = fp.assign_values_to_grid(dfs[sec],gridfile,'CenLon','CenLat',species)
    temp.to_netcdf(f'D:/Download/{secname}.nc')
    print(f'{sec} finished!')


Complete SO2
Complete NOx
Complete CO
Complete PM10
Complete PM2.5
Complete VOCs
Complete NH3
Complete BC
Complete OC
机动车 finished!
Complete SO2
Complete NOx
Complete CO
Complete PM10
Complete PM2.5
Complete VOCs
Complete NH3
Complete BC
Complete OC
非道路 finished!
Complete SO2
Complete NOx
Complete CO
Complete PM10
Complete PM2.5
Complete VOCs
Complete NH3
Complete BC
Complete OC
扬尘源 finished!
Complete SO2
Complete NOx
Complete CO
Complete PM10
Complete PM2.5
Complete VOCs
Complete NH3
Complete BC
Complete OC
非工业溶剂使用 finished!
Complete SO2
Complete NOx
Complete CO
Complete PM10
Complete PM2.5
Complete VOCs
Complete NH3
Complete BC
Complete OC
生物质 finished!
Complete SO2
Complete NOx
Complete CO
Complete PM10
Complete PM2.5
Complete VOCs
Complete NH3
Complete BC
Complete OC
农业源 finished!
Complete SO2
Complete NOx
Complete CO
Complete PM10
Complete PM2.5
Complete VOCs
Complete NH3
Complete BC
Complete OC
其他排放源 finished!
Complete SO2
Complete NOx
Complete CO
Complete PM10
Complete PM2.5
Com